In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from groq import Groq
from src.llm_engine.gemini_agent import Agent


In [2]:
df = pd.read_csv(r"./data/GridPuzzle_processed.csv")

In [2]:
# client = Groq(
#     api_key= os.getenv("GROQ_API_KEY"),
# )

client = Groq(api_key= os.getenv("GROQ_API_KEY"))

#### GROQ: Self-Refinement

In [4]:
path = "./deepseek_refinement_responses"

if not os.path.exists(path):
    os.mkdir(path)

In [6]:
ids_ =[]
for file in os.listdir(path):
    id_ = file.split("_")[-1].split(".")[0]
    ids_.append(int(id_))

ids_

[821,
 897,
 919,
 13,
 16,
 244,
 350,
 411,
 444,
 503,
 656,
 687,
 688,
 698,
 718,
 726,
 743,
 744,
 755,
 802]

In [7]:
from openai import OpenAI, APIStatusError
for i, row in df.iterrows():
    
    id_ = row["id"]
    if id_ not in ids_:
        user_query = row["question"]

        try:

            groq_deepseek_response = ""

            completion = client.chat.completions.create(
                model="deepseek-r1-distill-llama-70b",
                messages=[{
                        "role": "user",
                        "content": user_query

                    }],
                temperature=0,
                max_completion_tokens=8192,
                top_p=1,
                stream=True,
                stop=None,
            )

            for chunk in completion:
                text = chunk.choices[0].delta.content or ""
                groq_deepseek_response += text

            self_refinement = """Now that you've proposed a solution, critically review your work. Verify that you've properly interpreted all clues
            and reached a valid conclusion. If you identify any errors or inconsistencies in your reasoning, please reconsider the puzzle and provide
            a corrected solution."""

            new_query = self_refinement + "\n" + "Your Response: \n" + groq_deepseek_response
            return_seq = row["question"].split("Final Answer:")[1]
            new_query += return_seq + "\n"



            new_completion = client.chat.completions.create(
                model="deepseek-r1-distill-llama-70b",
                messages=[{
                        "role": "user",
                        "content": new_query

                    }],
                temperature=0,
                max_completion_tokens=8192,
                top_p=1,
                stream=True,
                stop=None,
            )

            final_response = ""
            for new_chunk in new_completion:
                text = new_chunk.choices[0].delta.content or ""
                final_response += text


            with open(os.path.join(path, f"deepseek_refinement_{id_}.txt"), "w") as f:
                f.write(final_response)
        
        except Exception as e:
            with open(os.path.join(path, f"deepseek_api_status_error_refinement_{id_}.txt"), "w") as f:
                f.write(final_response)

In [4]:
print(df["question"].iloc[0])

Nicholas and Edith are having a small dinner party this evening at their home in Cedar Valley, and they've decided to open a select number of rare wines from their personal collection to celebrate the occasion. Using only the clues below, match the vintages to the options from wines and types. Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.

vintages : 1984, 1988, 1992, 1996.
wines : Annata Branco, Bianca Flaux, Ece Suss, Vendemmia.
types : gewurztraminer, merlot, pinot noir, riesling.

Clues:
1. The Ece Suss was bottled sometime after the Annata Branco.
2. The Bianca Flaux was bottled 4 years before the Vendemmia.
3. The 1988 bottle is a pinot noir.
4. The merlot is either the Annata Branco or the Bianca Flaux.
5. The Bianca Flaux was bottled sometime after the Ece Suss.
6. The 1984 bottle is a gewurztraminer.

While answering use the following format:
Step-by-step solution:
Your steps showing how you are solving the puzzle
F

#### GEMINI Refinement

In [ ]:
path = "./gemini_refinement_responses"

if not os.path.exists(path):
    os.mkdir(path)


In [ ]:
agent = Agent(system_prompt="You are an AI assistant", agent_name="csp_agent")

for i, row in df.iterrows():
    
    id_ = row["id"]
    
    user_query = row["question"]

    response = agent.perform_action(user_query=user_query)

    try:
        self_refinement = """Now that you've proposed a solution, critically review your work. Verify that you've properly interpreted all clues
        and reached a valid conclusion. If you identify any errors or inconsistencies in your reasoning, please reconsider the puzzle and provide
        a corrected solution."""

        new_query = self_refinement + "\n" + "Your Response: \n" + response
        return_seq = row["question"].split("Final Answer:")[1]
        new_query += return_seq + "\n"

        final_response = agent.perform_action(user_query=new_query)


        with open(os.path.join(path, f"gemini_refinement_pred_{id_}.txt"), "w") as f:
            f.write(final_response)
    
    except Exception as e:
        with open(os.path.join(path, f"gemini_refinement_api_error_pred_{id_}.txt"), "w") as f:
            f.write(response)
